<a href="https://colab.research.google.com/github/GustavinhoDantas/Duplicidade-/blob/main/Duplicidade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DUPLICIDADES



In [ ]:
import pandas as pd

# Mude aqui para o nome do seu arquivo, com caminho correto no Colab
arquivo_entrada = '/content/SOCORRO.xlsx'  # pode ser .xlsx ou .csv

# Tentar ler Excel, se não conseguir, tenta CSV
try:
    df = pd.read_excel(arquivo_entrada, engine='openpyxl')
    print("Arquivo lido como Excel.")
except Exception as e:
    print(f"Erro lendo Excel: {e}\nTentando CSV...")
    df = pd.read_csv(arquivo_entrada)
    print("Arquivo lido como CSV.")

# Filtra linhas com STATUS diferente de 'Cancelado'
df_filtrado = df[df['STATUS'].str.strip().str.upper() != 'CANCELADO']

# Pega lista de apólices únicas da coluna 'ÚNICO APÓLICES', ignorando NaN
apolices_unicas = df_filtrado['ÚNICO APÓLICES'].dropna().unique()

# Função que retorna lista de endossos para uma apólice
def pega_endossos(apolice):
    # Filtra linhas onde 'APÓLICES SQL X RONALDO' == apolice e pega 'ENDOSSO RONALDO'
    endossos = df_filtrado.loc[df_filtrado['APÓLICES SQL X RONALDO'] == apolice, 'ENDOSSO RONALDO']
    return endossos.astype(str).tolist()

# Monta lista com resultados
resultado = []
for apolice in apolices_unicas:
    endossos_lista = pega_endossos(apolice)
    endossos_texto = ', '.join(endossos_lista)
    resultado.append({
        'Apólice': apolice,
        'Endossos': endossos_texto
    })

# Cria DataFrame resultado
df_resultado = pd.DataFrame(resultado)

# Salva arquivo novo Excel com as informações desejadas
arquivo_saida = '/content/resultado_endossos.xlsx'
df_resultado.to_excel(arquivo_saida, index=False, sheet_name='Endossos por Apolice')

print(f"Arquivo criado com sucesso: {arquivo_saida}")


Arquivo lido como Excel.
Arquivo criado com sucesso: /content/resultado_endossos.xlsx
